# Association Analysis 

#### Setup 

In [29]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [6]:
#import data 
df = pd.read_csv('nzlud_muni.csv')

In [9]:
df.head()

,GEOID,statename,place,type,restrict_sf_permit,restrict_mf_permit,limit_sf_units,limit_mf_units,limit_mf_dwellings,limit_mf_dwelling_units,min_lot_size,open_space,inclusionary,half_acre_less,half_acre_more,one_acre_more,two_acre_more,max_den_cat1,max_den_cat2,max_den_cat3,max_den_cat4,max_den_cat5,council_nz,planning_nz,countybrd_nz,pubhlth_nz,site_plan_nz,env_rev_nz,council_rz,planning_rz,zoning_rz,countybrd_rz,countyzone_rz,townmeet_rz,env_rev_rz,total_nz,total_rz,adu,height_ft_median,height_ft_mode,height_st_median,height_st_mode,parking_median,parking_mode,mf_per,maxden5,maxden4,maxden3,maxden2,maxden1,sindex1,sindex2,sindex3,sindex4,sindex5,sindex6,sindex7,sindex8,height_st_median_sc,height_st_mode_sc,sindex9,sindex10,sindex11,sindex9_st,sindex10_st,zri,zri_st,add_index,zri_up,zri_up_st,wt_all_2019,st_wt_all_2019,wt_msa_2019,st_wt_msa_2019,wt_lmsa_2019,st_wt_lmsa_2019,timestamp
0,102956,AL,Athens,City,0,0,0,0,0,0,1,0,0,1,0,0,0,0.0,0.0,0.0,1.0,0.0,1,1,0,0,0,0,1,1,0,0,0,0,0,2,2,1,12.0,10.0,2.0,2.0,2.00,2.0,0.285714,0,1,0,0,0,0,0,1,2,2,2,0,1,28.0,28.0,12.0,2.00,0.285714,-1.775183,0.164806,0.804929,-1.231126,8,0.881833,-0.765643,14.022902,0.886637,10.640125,1.206220,NaN,NaN,8/22/19 15:49
1,103076,AL,Auburn,City,0,0,0,0,0,0,1,1,0,0,0,0,1,0.0,0.0,1.0,0.0,0.0,1,1,0,0,0,0,1,1,0,0,0,0,0,2,2,1,35.0,45.0,2.0,2.0,2.00,2.0,0.058824,0,0,1,0,0,0,1,4,2,2,3,0,1,28.0,28.0,35.0,2.00,0.058824,0.049086,0.164806,2.152012,0.199853,13,1.919780,0.419004,2.132858,0.134856,2.491261,0.282422,NaN,NaN,8/22/19 15:50
2,105980,AL,Bessemer,City,0,0,0,0,0,0,1,1,0,1,0,0,0,0.0,0.0,0.0,0.0,1.0,1,1,0,0,1,0,1,1,1,0,0,0,0,3,3,0,35.0,35.0,NaN,NaN,4.25,1.0,0.285714,1,0,0,0,0,0,1,1,3,3,1,0,0,NaN,NaN,35.0,4.25,0.285714,0.049086,2.521087,2.217050,0.268941,9,1.174291,-0.431849,10.157370,0.642228,6.965198,0.789611,NaN,NaN,7/16/19 14:21
3,107000,AL,Birmingham,City,0,0,0,0,0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,1.0,0.0,1,1,0,0,1,0,1,1,1,0,0,0,0,3,3,0,45.0,45.0,3.0,3.0,3.00,1.0,0.428571,0,1,0,0,0,0,1,1,3,3,2,0,0,42.0,42.0,45.0,3.00,0.428571,0.842247,1.212042,1.871958,-0.097643,10,1.014156,-0.614617,2.120791,0.134093,2.194012,0.248725,NaN,NaN,7/16/19 14:21
4,114392,AL,Chickasaw,City,0,0,0,0,0,0,1,1,0,1,0,0,0,0.0,0.0,1.0,0.0,1.0,1,1,0,0,0,0,1,1,1,0,0,0,0,2,3,0,35.0,35.0,3.0,3.0,1.50,2.0,0.666667,1,0,0,0,0,0,1,1,2,3,1,0,0,42.0,42.0,35.0,1.50,0.666667,0.049086,-0.358812,1.821209,-0.151553,8,1.155308,-0.453515,13.094004,0.827904,8.369652,0.948827,NaN,NaN,10/18/19 23:07


In [11]:
#choose only binary data indicators
binary = df.iloc[:, np.r_[4:35, 37:38]]

In [13]:
#show summary to check for missing values 
binary.describe()

,restrict_sf_permit,restrict_mf_permit,limit_sf_units,limit_mf_units,limit_mf_dwellings,limit_mf_dwelling_units,min_lot_size,open_space,inclusionary,half_acre_less,half_acre_more,one_acre_more,two_acre_more,max_den_cat1,max_den_cat2,max_den_cat3,max_den_cat4,max_den_cat5,council_nz,planning_nz,countybrd_nz,pubhlth_nz,site_plan_nz,env_rev_nz,council_rz,planning_rz,zoning_rz,countybrd_rz,countyzone_rz,townmeet_rz,env_rev_rz,adu
count,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000,2639.000000
mean,0.019704,0.019704,0.025009,0.025009,0.019704,0.018189,0.999242,0.676014,0.286093,0.430087,0.140205,0.184161,0.245548,0.138415,0.120212,0.216913,0.305271,0.473265,0.805987,0.934824,0.014020,0.050398,0.201213,0.007579,0.913604,0.956044,0.718833,0.030315,0.018568,0.043198,0.004547,0.460781
std,0.139009,0.139009,0.156183,0.156183,0.139009,0.133659,0.027524,0.468084,0.452019,0.495182,0.347265,0.387689,0.430493,0.345400,0.325271,0.412221,0.460610,0.499379,0.395514,0.246883,0.117597,0.218806,0.400983,0.086741,0.281002,0.205036,0.449654,0.171484,0.135018,0.203342,0.067292,0.498554
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
#choose only "transactions" without null values
binary_clean = binary.dropna()

In [17]:
#double check that each basket item is in binary format 
for column in binary_clean.columns:
    unique_vals = binary_clean[column].unique()
    print(f"{column}: {unique_vals}")

restrict_sf_permit: [0 1]
restrict_mf_permit: [0 1]
limit_sf_units: [0 1]
limit_mf_units: [0 1]
limit_mf_dwellings: [0 1]
limit_mf_dwelling_units: [0 1]
min_lot_size: [1 0]
open_space: [0 1]
inclusionary: [0 1]
half_acre_less: [1 0]
half_acre_more: [0 1]
one_acre_more: [0 1]
two_acre_more: [0 1]
max_den_cat1: [0. 1.]
max_den_cat2: [0. 1.]
max_den_cat3: [0. 1.]
max_den_cat4: [1. 0.]
max_den_cat5: [0. 1.]
council_nz: [1 0]
planning_nz: [1 0]
countybrd_nz: [0 1]
pubhlth_nz: [0 1]
site_plan_nz: [0 1]
env_rev_nz: [0 1]
council_rz: [1 0]
planning_rz: [1 0]
zoning_rz: [0 1]
countybrd_rz: [0 1]
countyzone_rz: [0 1]
townmeet_rz: [0 1]
env_rev_rz: [0 1]
adu: [1 0]


## Perform Association Analysis

In [39]:
# Generate frequent itemsets
frequent_itemsets = apriori(binary_clean, min_support=0.2, use_colnames=True)
    
# Generate association rules
rules = association_rules(frequent_itemsets, 
                           metric="confidence", 
                           min_threshold=0.5,
                         num_itemsets = len(binary_clean.columns)
                         )

/Users/aasthajha/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [43]:
negative_association = rules[rules['lift'] < .87].sort_values('lift')
negative_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
21,(half_acre_less),(open_space),0.430413,0.675768,0.249905,0.580617,0.859195,1.0,-0.040954,0.773116,-0.223432,0.291851,-0.293466,0.475213
93,(half_acre_less),"(open_space, min_lot_size)",0.430413,0.675768,0.249905,0.580617,0.859195,1.0,-0.040954,0.773116,-0.223432,0.291851,-0.293466,0.475213
92,"(half_acre_less, min_lot_size)",(open_space),0.429655,0.675768,0.249905,0.581642,0.860712,1.0,-0.040442,0.775010,-0.221025,0.292110,-0.290305,0.475725
308,(half_acre_less),"(planning_rz, open_space)",0.430413,0.653773,0.242700,0.563877,0.862496,1.0,-0.038693,0.793874,-0.218688,0.288418,-0.259647,0.467553
685,(half_acre_less),"(planning_rz, open_space, min_lot_size)",0.430413,0.653773,0.242700,0.563877,0.862496,1.0,-0.038693,0.793874,-0.218688,0.288418,-0.259647,0.467553
302,(half_acre_less),"(open_space, planning_nz)",0.430413,0.645810,0.240046,0.557709,0.863582,1.0,-0.037920,0.800809,-0.217122,0.287075,-0.248737,0.464703
671,(half_acre_less),"(open_space, planning_nz, min_lot_size)",0.430413,0.645810,0.240046,0.557709,0.863582,1.0,-0.037920,0.800809,-0.217122,0.287075,-0.248737,0.464703
684,"(half_acre_less, min_lot_size)","(planning_rz, open_space)",0.429655,0.653773,0.242700,0.564872,0.864018,1.0,-0.038197,0.795690,-0.216266,0.288678,-0.256772,0.468051
1495,(half_acre_less),"(planning_rz, open_space, planning_nz)",0.430413,0.635571,0.236633,0.549780,0.865017,1.0,-0.036926,0.809447,-0.215048,0.285322,-0.235412,0.461047
2304,(half_acre_less),"(planning_rz, open_space, min_lot_size, planning_nz)",0.430413,0.635571,0.236633,0.549780,0.865017,1.0,-0.036926,0.809447,-0.215048,0.285322,-0.235412,0.461047


In [47]:
positive_association = rules[(rules['lift'] > 1.3) & (rules['confidence'] > 0.7) ].sort_values('lift')
positive_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
6347,"(adu, council_rz, planning_nz)","(planning_rz, open_space, council_nz)",0.395904,0.538111,0.278347,0.703065,1.306542,1.0,0.065306,1.555521,0.388383,0.424523,0.357129,0.610165
6850,"(adu, council_rz, min_lot_size, planning_nz)","(planning_rz, open_space, council_nz)",0.395904,0.538111,0.278347,0.703065,1.306542,1.0,0.065306,1.555521,0.388383,0.424523,0.357129,0.610165
6868,"(adu, council_rz, planning_nz)","(planning_rz, open_space, council_nz, min_lot_size)",0.395904,0.538111,0.278347,0.703065,1.306542,1.0,0.065306,1.555521,0.388383,0.424523,0.357129,0.610165
431,(inclusionary),"(planning_rz, adu)",0.286310,0.445582,0.200986,0.701987,1.575438,1.0,0.073411,1.860380,0.511785,0.378571,0.462475,0.576525
964,"(inclusionary, min_lot_size)","(planning_rz, adu)",0.286310,0.445582,0.200986,0.701987,1.575438,1.0,0.073411,1.860380,0.511785,0.378571,0.462475,0.576525
965,(inclusionary),"(planning_rz, adu, min_lot_size)",0.286310,0.445582,0.200986,0.701987,1.575438,1.0,0.073411,1.860380,0.511785,0.378571,0.462475,0.576525
429,"(planning_rz, inclusionary)",(adu),0.275313,0.461130,0.200986,0.730028,1.583127,1.0,0.074031,1.996018,0.508273,0.375354,0.499003,0.582941
960,"(planning_rz, inclusionary, min_lot_size)",(adu),0.275313,0.461130,0.200986,0.730028,1.583127,1.0,0.074031,1.996018,0.508273,0.375354,0.499003,0.582941
962,"(planning_rz, inclusionary)","(adu, min_lot_size)",0.275313,0.461130,0.200986,0.730028,1.583127,1.0,0.074031,1.996018,0.508273,0.375354,0.499003,0.582941
40,(inclusionary),(adu),0.286310,0.461130,0.209329,0.731126,1.585509,1.0,0.077303,2.004171,0.517434,0.389006,0.501041,0.592537


In [559]:
rules_df = pd.concat([positive_association, negative_association])